In [2]:
from pyspark.sql import SparkSession

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("DataCleaning")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.driver.memory', '4g')
    .config('spark.executor.memory', '2g')
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
24/08/17 17:29:17 WARN Utils: Your hostname, Bobert resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/08/17 17:29:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/17 17:29:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
from pyspark.sql.functions import min,max
from pyspark.sql import functions as F
import pandas as pd
import geopandas as gpd
from pyspark.sql.functions import col, when, unix_timestamp
import math
import pandas as pd
from pyspark.sql.types import IntegerType, datetime

In [ ]:
# Checking the data for the Yellow Taxi dataset

# check passenger_count, trip_distance, PU/DOLocationID, payment_type, fare_amount, extra, mta_tax, tip_amount, tolls_amount
#, improvement_surcharge, songestion_surcharge

ytaxi = spark.read.parquet('../data/curated/YellowTaxi_Checkpt1')

In [ ]:
# NOTES FOR SELF

# Yellow Taxi
''' max trip distance is way too long, new york has 320sq miles of land, remove some entries

    9 passenger rides are not legal and would not fit in a cab, the max allowed is 7 on cases where there is a child

    min and max fare amounts are ridiculous, adjust them

    extra min max are insane

    negative mta_tax, also max is 42, maybe that's okay?

    negative tip amount not possible, max is 4000 which is not right

    negative tolls, max tolls

    pos and neg improvement and congestion charges, these are probably meant to all be pos
'''

In [ ]:
# Exploratory Analysis for Yellow Taxi dataset

check_columns = ['passenger_count', 'trip_distance', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount'
, 'improvement_surcharge', 'congestion_surcharge']

for column in check_columns:
    ytaxi.agg(min(column),max(column)).show()

In [ ]:
# Cleaning

ytaxi_clean = ytaxi
ytaxi_clean.where(F.col('trip_distance')>300)

ytaxi.describe('trip_distance')
#ytaxi.stat.approxQuantile('trip_distance', [0.9999],0.00001)
#ytaxi.where(F.col('trip_distance')>55).count()
#ytaxi.where(F.col('trip_distance')<0).count()

ytaxi_clean = ytaxi.where((F.col('trip_distance')<55)&(F.col('trip_distance')>0.65)&(F.col('passenger_count')<=7)&
                          (F.col('passenger_count')>=0)&(F.col('fare_amount')>2.5)&(F.col('tip_amount')>=0)
                          &(F.col('tip_amount')<65)&(F.col('payment_type')!=2)&(F.col('RatecodeID')<=6)
                          &(F.col('fare_amount')<250))


(ytaxi_clean.count()*100)/ytaxi.count() # keeping 67.89% of original data as rest is corrupted

#ytaxi_clean.approxQuantile('tip_amount',[0.9999],0.00001) ---65
#ytaxi_clean.describe()


In [ ]:
ytaxi_clean.describe('extra') # website says it should only include values of 0.5 and 1? 

In [ ]:
# Creating a column to check how long the trips were
ytaxi_clean = ytaxi_clean.withColumn(
    "trip_duration", 
    (unix_timestamp(col("tpep_dropoff_datetime")) - unix_timestamp(col("tpep_pickup_datetime"))) / 60
)

#ytaxi_clean.where(F.col('diff_minutes')==0).count()
#ytaxi_clean.describe('diff_minutes')
#ytaxi_clean.approxQuantile('diff_minutes',[0.0001,0.9999],0.00001) # 0.02, 1435

ytaxi_clean.where(F.col('trip_duration')<=0.2).count()


In [ ]:
# average speed in new york is 4.5 mph

ytaxi_clean = ytaxi_clean.withColumn(
    'average_speed',
    (col('trip_distance')/(col('trip_duration')/60))
)

# ytaxi_clean.approxQuantile('average_speed',[0.25,0.5,0.75],0.00001) #50, some more leadway is 60

In [ ]:
ytaxi_clean = ytaxi_clean.where((col('average_speed')>4.4)&(col('average_speed')<65))

ytaxi_clean = ytaxi_clean.where(col('trip_duration')>1)

(ytaxi_clean.count()*100)/ytaxi.count()

In [ ]:
# Dropping the columns that are irrelevant for the analysis
ytaxi_clean = ytaxi_clean.drop('store_and_fwd_flag', 'extra', 'VendorID', 'mta_tax')


In [ ]:
# Making sure all the entries are within the intended date range
ytaxi_clean = ytaxi_clean.where((F.col('tpep_pickup_datetime')>='2023-12-01 00:00:00')&(F.col('tpep_pickup_datetime')<='2024-05-31 23:59:59')
                          &(F.col('tpep_dropoff_datetime')>='2023-12-01 00:00:00')&(F.col('tpep_dropoff_datetime')<='2024-05-31 23:59:59'))

# Adding in some more useful columns 

ytaxi_clean = ytaxi_clean.withColumn('pickup_day', F.date_format('tpep_pickup_datetime', 'E'))

ytaxi_clean = ytaxi_clean.withColumn('pickup_hour', F.date_format('tpep_pickup_datetime', 'H'))
ytaxi_clean = ytaxi_clean.withColumn('pickup_hour', ytaxi_clean['pickup_hour'].cast(IntegerType()))

In [ ]:
ytaxi_clean.write.mode('overwrite').parquet('../data/curated/YellowTaxi_Cleaned')

In [ ]:
# Cleaning the weather dataset

weather = pd.read_csv('../data/landing/new york 2023-12-01 to 2024-05-31.csv')

# dropping columns that are not realistic to check/impact days of everyday citizens or are irrelevant such as 
# stations, source, name, moonphase, 
weather.drop(['stations', 'name','solarradiation','solarenergy','precipprob','preciptype'],axis=1,inplace=True)

weather.describe()
# no further cleaning required, dataset seems clean

weather.to_csv('../data/curated/Weather/WeatherClean.csv',index=False)

weather.to_parquet('../data/curated/Weather/WeatherCleaned')

In [ ]:
ytaxi_clean.count()*100/ytaxi.count() # 64.98% of the original data is kept

In [28]:
# Combining the weather and ytaxi_clean datasets

weather = spark.read.parquet('../data/curated/Weather/WeatherCleaned')
ytaxi_clean = spark.read.parquet('../data/curated/YellowTaxi_Cleaned')
# Converting datatype of datetime in weather to datetime type instead of string

weather = weather.withColumn('datetime', F.regexp_replace('datetime', 'T', ' '))
weather = weather.withColumn('datetime', F.to_timestamp('datetime', 'yyyy-MM-dd HH:mm:ss'))

from pyspark.sql.functions import date_trunc

ytaxi_clean = ytaxi_clean.withColumn('tpep_pickup_datetime', date_trunc('hour', 'tpep_pickup_datetime'))
ytaxi_clean = ytaxi_clean.withColumn('tpep_dropoff_datetime', date_trunc('hour', 'tpep_dropoff_datetime'))

joined_df = ytaxi_clean.join(weather, ytaxi_clean.tpep_pickup_datetime == weather.datetime, "left")

joined_df.write.parquet('../data/curated/YellowTaxi_Weather')